In [ ]:
from itertools import chain
from matplotlib import cm
import matplotlib.pyplot as plt
import toml
from tqdm import tqdm
import numpy as np
import cmasher as cmr
from typing import Union, List, Dict 
import cartopy.crs as ccrs
from glob import glob
import pandas as pd
import cartopy as cp

def project_points(
    projection,
    lon: Union[np.ndarray, float],
    lat: Union[np.ndarray, float],
):
    """
    Define the correct projection function depending on name of projection
    :param projection: Projection to be used
    :type projection: cp.crs.Projection
    :param lon: Longitude coordinate
    :type lon: Union[np.ndarray, float]
    :param lat: Latitude coordinate
    :type lat: Union[np.ndarray, float]
    :return: projected lon, lat points
    :rtype: np.ndarray, np.ndarray
    """
    import pyproj
    import cartopy as cp

    proj_dict = projection.proj4_params

    # projection = pyproj.crs.CRS.from_dict(proj_dict)
    event_loc = pyproj.Proj(proj_dict, preserve_units=True)
    x, y = event_loc(lon, lat)
    if isinstance(x, list):
        x = np.array(x)
        y = np.array(y)
    return x, y
def plot_raydensity(
    map_object,
    station_events: list,
    projection,
):
    """
    Create a ray-density plot for all events and all stations.
    This function is potentially expensive and will use all CPUs available.
    Does require geographiclib to be installed.
    :param map_object: The cartopy domain plot object
    :type map_object: cp.mpl.geoaxes.GeoAxes
    :param station_events: A list of tuples with two dictionaries
    :type station_events: List[Tuple[dict, dict]]
    :param projection: cartopy projection object
    :type projection: cp.crs.Projection
    """
    import ctypes as C
    from lasif.tools.great_circle_binner import GreatCircleBinner
    from lasif.utils import Point
    import multiprocessing
    import progressbar
    from scipy.stats import scoreatpercentile
    import cartopy as cp

    # Merge everything so that a list with coordinate pairs is created. This
    # list is then distributed among all processors.
    station_event_list = []
    for event, station in station_events:
      #  print(event)
      #  print(station)
        e_point = Point(event["latitude"], event["longitude"])
        #for station in stations.values():
        p = Point(station["latitude"], station["longitude"])
        station_event_list.append((e_point, p))

    circle_count = len(station_event_list)

    # The granularity of the latitude/longitude discretization for the
    # raypaths. Attempt to get a somewhat meaningful result in any case.
    if circle_count < 1000:
        lat_lng_count = 1000
    elif circle_count < 10000:
        lat_lng_count = 2000
    else:
        lat_lng_count = 3000

    cpu_count = multiprocessing.cpu_count()

    def to_numpy(raw_array, dtype, shape):
        data = np.frombuffer(raw_array.get_obj())
        data.dtype = dtype
        return data.reshape(shape)

    print(
        "\nLaunching %i great circle calculations on %i CPUs..."
        % (circle_count, cpu_count)
    )

    widgets = [
        "Progress: ",
        progressbar.Percentage(),
        progressbar.Bar(),
        "",
        progressbar.ETA(),
    ]
    pbar = progressbar.ProgressBar(
        widgets=widgets, maxval=circle_count
    ).start()

    def great_circle_binning(
        sta_evs, bin_data_buffer, bin_data_shape, lock, counter
    ):
        new_bins = GreatCircleBinner(
            -89.,
            89.,
            lat_lng_count,
            -180.,
            180.,
            lat_lng_count,
        )
        for event, station in sta_evs:
            with lock:
                counter.value += 1
            if not counter.value % 25:
                pbar.update(counter.value)
            new_bins.add_greatcircle(event, station)

        bin_data = to_numpy(bin_data_buffer, np.uint32, bin_data_shape)
        with bin_data_buffer.get_lock():
            bin_data += new_bins.bins

    # Split the data in cpu_count parts.
    def chunk(seq, num):
        avg = len(seq) / float(num)
        out = []
        last = 0.0
        while last < len(seq):
            out.append(seq[int(last) : int(last + avg)])
            last += avg
        return out

    chunks = chunk(station_event_list, cpu_count)

    # One instance that collects everything.
    collected_bins = GreatCircleBinner(
        -89.,
        89.,
        lat_lng_count,
        -180.,
        180.,
        lat_lng_count,
    )

    # Use a multiprocessing shared memory array and map it to a numpy view.
    collected_bins_data = multiprocessing.Array(
        C.c_uint32, collected_bins.bins.size
    )
    collected_bins.bins = to_numpy(
        collected_bins_data, np.uint32, collected_bins.bins.shape
    )

    # Create, launch and join one process per CPU. Use a shared value as a
    # counter and a lock to avoid race conditions.
    processes = []
    lock = multiprocessing.Lock()
    counter = multiprocessing.Value("i", 0)
    for _i in range(cpu_count):
        processes.append(
            multiprocessing.Process(
                target=great_circle_binning,
                args=(
                    chunks[_i],
                    collected_bins_data,
                    collected_bins.bins.shape,
                    lock,
                    counter,
                ),
            )
        )
    for process in processes:
        process.start()
    for process in processes:
        process.join()

    pbar.finish()

    stations = chain.from_iterable(
        (_i[1].values() for _i in station_events if _i[1])
    )
    # Remove duplicates
    #stations = [(_i["latitude"], _i["longitude"]) for _i in stations]
    #stations = set(stations)
    #title = "%i Events, %i unique raypaths, " "%i unique stations" % (
    #    len(station_events),
    #    circle_count,
    #    len(stations),
    #)
    #plt.title(title, size="xx-large")

    data = collected_bins.bins.transpose()

    if data.max() >= 10:
        data = np.log10(np.clip(data, a_min=0.5, a_max=data.max()))
        data[data >= 0.0] += 0.1
        data[data < 0.0] = 0.0
        max_val = scoreatpercentile(data.ravel(), 99.99)
    else:
        max_val = data.max()

    cmap = cm.get_cmap("gist_heat")
#     cmap = cm.get_cmap("gist_gray_r")
    cmap._init()
    cmap._lut[:120, -1] = np.linspace(0, 1.0, 120) ** 2

    lngs, lats = collected_bins.coordinates
    ln, la = project_points(projection, lngs, lats)

    map_object.pcolormesh(
        ln, la, data, cmap=cmap, vmin=0, vmax=max_val, zorder=10,# alpha = 1.0
    )
    # Draw the coastlines so they appear over the rays. Otherwise things are
    # sometimes hard to see.
#     map_object.add_feature(cp.feature.COASTLINE, zorder=13)
#     map_object.add_feature(cp.feature.BORDERS, linestyle=":", zorder=13)
    
    return map_object

In [ ]:
# Create list with information for rayplot
datalist = glob('./coordinate_info/*.toml') 
# Where to save
savepath = '/path/to/savedir'
# Resolution
dpi = 300

station_events = []
for event in tqdm(datalist):
    d = toml.load(event)
    for j, rec in enumerate(d['receiver_coords']):
        #if j%500!=0: continue
        station_events.append(({'longitude':d['src_lon'], 
                                 'latitude':d['src_lat']},
                               {'longitude':d['receiver_coords'][rec]['longitude'], 
                                'latitude':d['receiver_coords'][rec]['latitude']}))

In [ ]:
# Global Figure
fig = plt.figure(figsize=(12, 6))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.Robinson(central_longitude=0))
ax = plot_raydensity(ax, station_events, projection = ccrs.Robinson(central_longitude=0))
plt.savefig(savepath,  bbox_inches='tight', dpi = dpi)
plt.show()